In [114]:
import requests
import pickle
import time
import pandas as pd
from api import SearchListings, ListingDetails

In [140]:
toplevel_props = ['ListingID', 'DefaultParentArea', 'Zip', 'Url', 'Building', 'BathsFull', 'BathsHalf', 'Beds', 'CurrentPrice',\
    'Days', 'FullStreetAddress', 'Latitude', 'Longitude', 'ListDate', 'LotSize', 'OriginalPrice', 'Ownership',\
    'PropertyStyle', 'PropertyType', 'Remarks', 'SqFt', 'TownhouseType', 'UnitCount', 'WalkScore', 'WalkScoreDescription', 'YearBuilt',\
    'Zip', 'Details', 'Amenities', 'BuildingAmenities']
secondlevel_props = {'DefaultParentArea': [['Name', 'Neighborhood']],\
    'Building': [['UnitCount', 'BuildingUnitCount'],['Name', 'BuildingName']]}
details_props = {
    'Interior Features': ['Accessibility Features', 'AllRoom Features', 'Elevator', 'Interior Features',\
        'Kitchen Appliances', 'Master Bedroom', 'Bedroom 2', 'Dining Room', 'Living Room'],\
    'Utilities': ['Fuel Description', 'Hot Water', 'Sewer', 'Water Description', 'Cooling Description',\
        'Heating Description'],\
    'Green Features': ['Green Energy Supplement', 'Green Verification HES Metric', 'Green Verification HES Year', "Green Verification HES "]
}

In [141]:
class HomeScraper():
    def __init__(self, city):
        self.city = city
        self.search_api = SearchListings(city)
        self.listings_api = ListingDetails()
        self.listing_detail_cookies = None
        self.total_listings = 0
        self.listings = None

    def download_listings(self):
        url = self.search_api.url()
        headers = self.search_api.headers()

        #use get to pull cookies
        get_response = requests.get(url, headers=headers)
        assert get_response.status_code == 200, f"failed get request with status code: {get_response.status_code}"


        #get the count of active listings
        time.sleep(1)
        payload = self.search_api.payload(maximumListings=10)
        post_response = requests.post(url, cookies=get_response.cookies, headers=headers, json=payload)
        assert post_response.status_code == 200, f'post request to search listings returned w/ status code: {post_response.status_code}'
        response_json = post_response.json()
        self.total_listings = response_json['d']['Count']

        # get all active listings
        time.sleep(1)
        payload = self.search_api.payload(maximumListings=self.total_listings)
        post_response = requests.post(url, cookies=get_response.cookies, headers=headers, json=payload)
        assert post_response.status_code == 200, f'post request to search listings returned w/ status code: {post_response.status_code}'
        response_json = post_response.json()
        self.listings = response_json['d']['Listings']
        return
    
    def download_listing_details(self, listing_id):
        # download detail of a single listing
        url = self.listings_api.url()
        headers = self.listings_api.headers()
        payload = self.listings_api.payload(listing_id)
        
        # TODO: use get to pull cookies
        # get_response = requests.get(url, headers=headers)
        # assert get_response.status_code == 200, f"failed get request with status code: {get_response.status_code}"

        # get details
        post_response = requests.post(url, cookies=self.listing_detail_cookies, headers=headers, json=payload)
        assert post_response.status_code == 200, f'post request to listing details returned w/ status code: {post_response.status_code}'
        if not self.listing_detail_cookies:
            self.listing_detail_cookies = post_response.cookies
        response_json = post_response.json()
        return response_json['d']
    
    def download_details(self, count=None):
        if not count:
            count = self.count
        # download details of all listings
        all_listings_details = []
        for listing in self.listings[:count]:
            details = self.download_listing_details(listing['Listing']['ID'])
            time.sleep(1)
            all_listings_details.append(details)
        return all_listings_details
    
    def to_pickle(self):
        # store as object to local filesystem
        filename = f'{self.city}_homesnap_scraper.pickle'
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
        return



        

    
    

In [142]:
home_scraper = HomeScraper('Portland, OR')
home_scraper.download_listings()

In [143]:
home_scraper.to_pickle()

In [144]:
with open('Portland, OR_homesnap_scraper.pickle', 'rb') as f:
    portland_scraper = pickle.load(f) 

In [145]:
all_listings_details = portland_scraper.download_details(2)
all_listings_details

[{'__type': 'Homesnap.API.HSListingDetail',
  'DefaultParentAreaID': 109799,
  'DefaultParentArea': {'AreaID': 109799,
   'Name': 'Pearl',
   'ShortName': 'Pearl',
   'Subhead': 'Portland, OR Neighborhood',
   'State': 'OR',
   'USPSCity': 'Portland',
   'AreaTypeID': 6,
   'SubTypeID': 2,
   'TopLevelAreaID': 372,
   'CountyID': 3234,
   'Polygon': None,
   'Levels': None,
   'North': 45.537983,
   'South': 45.522902,
   'East': -122.67419,
   'West': -122.687117,
   'HasValidCoordinates': True,
   'MostRecentListDate': '/Date(1648471788000)/',
   'Url': '/sitemap/OR/Portland/Pearl',
   'ImageUrl': 'https://s3.amazonaws.com/homesnap.areas/0/ni/6/original.jpg',
   'UrlForSale': '/homes/for_sale/OR/Portland/Pearl/p_21,109799',
   'UrlForSaleOpenHouse': '/homes/for_sale/open_house/OR/Portland/Pearl/p_21,109799/f_1',
   'UrlForRent': '/homes/for_rent/OR/Portland/Pearl/p_21,109799',
   'Status': 15,
   'CityModeID': 0,
   'AreaTypeFlag': 32,
   'HasMLSCoverage': True,
   'EntityType': 21,


In [146]:
def clean_listing_details(listings_details):
    cleaned_listings_details = [] 
    for details in listings_details:
        details_dict = {}
        for prop in toplevel_props:
            inner_details = details[prop]
            if not inner_details:
                details_dict[prop] = None
            elif prop in secondlevel_props.keys():
                # for nested properties, extract the values
                # and store in a renamed property key
                for inner_prop in secondlevel_props[prop]:
                    details_dict[inner_prop[1]] = inner_details[inner_prop[0]]
            elif prop == 'Details':
                # extract more deeply nested values
                for inner_prop in inner_details:
                    inner_prop_name = inner_prop['Name']
                    if inner_prop_name in details_props.keys():
                        for innermost_prop in inner_prop['Fields']:
                            innermost_prop_name = innermost_prop['Name']
                            if innermost_prop_name in details_props[inner_prop_name]:
                                details_dict[innermost_prop_name] = innermost_prop['Value']
            else:
                details_dict[prop] = inner_details 
            
        cleaned_listings_details.append(details_dict)
    return pd.json_normalize(cleaned_listings_details)

In [147]:
cleaned_details = clean_listing_details(all_listings_details)

In [149]:
cleaned_details.head()

,ListingID,Neighborhood,Zip,Url,BuildingUnitCount,BuildingName,BathsFull,BathsHalf,Beds,CurrentPrice,Days,FullStreetAddress,Latitude,Longitude,ListDate,LotSize,OriginalPrice,Ownership,PropertyStyle,PropertyType,Remarks,SqFt,TownhouseType,UnitCount,WalkScore,WalkScoreDescription,YearBuilt,Elevator,Interior Features,Kitchen Appliances,Master Bedroom,Fuel Description,Sewer,Water Description,Heating Description,Green Energy Supplement,Amenities,BuildingAmenities,Building,Accessibility Features,AllRoom Features,Bedroom 2,Dining Room,Living Room,Hot Water
0,118203256,Pearl,97209,/OR/Portland/1030-NW-12th-Avenue/property/3000...,139.0,NaN,1,None,NaN,275000,None,1030 NW 12th Avenue #324,45.530377,-122.683216,/Date(1648483200000)/,NaN,275000,None,"Contemporary, Loft","Residential,Condominium",INVESTORS! Nice open outlook to north. Has bee...,663,None,None,98,Walkers' Paradise,2001,Yes,"High Ceilings, 3rd Floor","Microwave, Dishwasher, Disposal, Free-Standing...",On Level: Main,"Electricity, Gas",Public Sewer,Public Water,Forced Air,No,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,118178944,Mt. Scott - Arleta,97206,/OR/Portland/6036-SE-Mitchell-Street,NaN,NaN,2,None,2.0,525000,None,6036 SE Mitchell Street,45.485195,-122.601135,/Date(1648483200000)/,0.1,525000,None,Bungalow,"Residential,Single Family Residence",Come see this charming & cozy two bedroom/two ...,1394,None,None,-1,None,1920,NaN,"Tile Floor, Vaulted Ceiling(s), Washer/Dryer, ...","Dishwasher, Tile, Free-Standing Refrigerator, ...","Features: Bathroom, Bay Window, Skylight(s), B...",Gas,Public Sewer,Public Water,Forced Air,NaN,None,None,NaN,"One Level, Walk-in Shower","Bathroom, Bay Window, Gas Appliances, Skylight...",On Level: Main,"Features: Bamboo Floor,On Level: Main","Features: Skylight(s), Vaulted Ceiling(s), Bam...",Gas


In [138]:
def store_all_listings(filename, data):
    # store to local system as pickle file
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
        return

store_all_listings('portland_or_listings_df.pickle', cleaned_details)

with open('portland_or_listings_df.pickle', 'rb') as f:
    portland_listings = pickle.load(f)

In [139]:
portland_listings

,ListingID,Neighborhood,Url,Building,BathsFull,BathsHalf,Beds,CurrentPrice,Days,FullStreetAddress,Latitude,Longitude,ListDate,LotSize,OriginalPrice,Ownership,PropertyStyle,PropertyType,Remarks,SqFt,TownhouseType,UnitCount,WalkScore,WalkScoreDescription,YearBuilt,Zip,Amenities,BuildingAmenities,BuildingUnitCount,BuildingName,Accessibility Features,AllRoom Features,Elevator,Interior Features,Kitchen Appliances,Master Bedroom,Bedroom 2,Dining Room,Living Room,Fuel Description,Hot Water,Sewer,Water Description,Cooling Description,Heating Description,Green Energy Supplement
0,118112620,Powellhurst - Gilbert,/OR/Portland/12520-SE-Holgate-Boulevard,NaN,NaN,None,NaN,399900,None,12520 SE Holgate Boulevard,45.488995,-122.534103,/Date(1648396800000)/,0.91,399900,None,None,"Lots and Land,Residential/Recreational",Tons of potential and charm in Powellhurst-Gil...,NaN,None,None,-1,None,NaN,97236,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,118116711,Downtown Portland,/OR/Portland/1500-SW-5th-Avenue/property/30001...,NaN,2.0,None,2.0,350000,None,1500 SW 5th Avenue #203,45.513192,-122.680291,/Date(1648396800000)/,NaN,350000,None,Contemporary,"Residential,Condominium","Highly desirable 2nd floor condo unit, next to...",1319.0,None,None,98,Walkers' Paradise,1973.0,97201,None,None,149.0,NaN,"One Level, Accessible Entrance, Main Floor Bed...","Gourmet Kitchen, Great Room, Suite, Wall to Wa...",Yes,"Garage Door Opener, Laundry, Washer/Dryer, Wal...","Microwave, Dishwasher, Disposal, Free-Standing...","Features: Suite, Wall to Wall Carpet,On Level:...",On Level: Main,"Features: Great Room, Wood Floors,On Level: Main","Features: Great Room, High Ceilings, Wood Floo...",Electricity,Electricity,Public Sewer,Public Water,Heat Pump,"Forced Air, Heat Pump",No
